In [1]:
%reload_ext autoreload
%autoreload 2
    
import navsim
from navsim.util import ObjDict, s_hwc_to_chw, ResourceCounter
from navsim import NavSimGymEnv, CupyMemory, NumpyMemory
import cupy as cp
import numpy as np

In [2]:
cp.show_config()

OS                           : Linux-5.8.0-59-generic-x86_64-with-glibc2.10
Python Version               : 3.8.10
CuPy Version                 : 9.1.0
CuPy Platform                : NVIDIA CUDA
NumPy Version                : 1.21.0
SciPy Version                : 1.7.0
Cython Build Version         : 0.29.23
Cython Runtime Version       : 0.29.23
CUDA Root                    : /opt/conda/envs/navsim
nvcc PATH                    : None
CUDA Build Version           : 11010
CUDA Driver Version          : 11020
CUDA Runtime Version         : 11010
cuBLAS Version               : 11300
cuFFT Version                : 10300
cuRAND Version               : 10202
cuSOLVER Version             : (11, 0, 1)
cuSPARSE Version             : 11300
NVRTC Version                : (11, 1)
Thrust Version               : 100910
CUB Build Version            : 100910
Jitify Build Version         : <unknown>
cuDNN Build Version          : 8100
cuDNN Version                : 8200
NCCL Build Version           : 209

In [3]:
print(cp.cuda.device.get_device_id())

0


In [4]:
env_config = ObjDict({
        "episode_max_steps":10,
        "task": 0,
        "goal": 0,
        "goal_distance": 50,
        "reward_for_goal": 50,
        "reward_for_ep": 0.005,
        "reward_for_other": -0.1,
        "reward_for_falling_off_map": -50,
        "reward_for_step": -0.0001,
        "agent_car_physics": 0,
        "env_path": "/data/work/unity-envs/Build2.8.1/Berlin_Walk_V2.x86_64",
        "debug":False,
        "run_base_folder_str":".",
        "observation_mode":0,
        "seed":123
    })

In [5]:
# test for seeds for reset() and action samples

num_episodes=10
num_steps=10
num_actions = 3

for seed in [123,321]:
    env_config["seed"]=seed
    reset_obs=cp.full((num_episodes,13),0.0)
    actions=cp.full((num_episodes,num_steps,3),0.0)
    for i in range(0,num_episodes):
        env = NavSimGymEnv(env_config)
        reset_obs[i,:] = cp.array(env.reset()[0])
        for j in range(0,num_steps):
            #for k in range(0,num_actions)
            actions[i,j,:]=cp.array(env.action_space.sample())
        env.close()
        del env
    print(reset_obs)
    print(actions)

[INFO] Connected to Unity environment with package version 2.1.0-exp.1 and communication version 1.5.0
Created UnityEnvironment at port 5005
[INFO] Connected new brain: VectorNavigator?team=0


/opt/conda/envs/navsim/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


[INFO] Connected to Unity environment with package version 2.1.0-exp.1 and communication version 1.5.0
Created UnityEnvironment at port 5005
[INFO] Connected new brain: VectorNavigator?team=0
[INFO] Connected to Unity environment with package version 2.1.0-exp.1 and communication version 1.5.0
Created UnityEnvironment at port 5005
[INFO] Connected new brain: VectorNavigator?team=0
[INFO] Connected to Unity environment with package version 2.1.0-exp.1 and communication version 1.5.0
Created UnityEnvironment at port 5005
[INFO] Connected new brain: VectorNavigator?team=0
[INFO] Connected to Unity environment with package version 2.1.0-exp.1 and communication version 1.5.0
Created UnityEnvironment at port 5006
[INFO] Connected new brain: VectorNavigator?team=0
[INFO] Connected to Unity environment with package version 2.1.0-exp.1 and communication version 1.5.0
Created UnityEnvironment at port 5006
[INFO] Connected new brain: VectorNavigator?team=0
[INFO] Connected to Unity environment wi

Created UnityEnvironment at port 5005
[INFO] Connected new brain: VectorNavigator?team=0
[INFO] Connected to Unity environment with package version 2.1.0-exp.1 and communication version 1.5.0
Created UnityEnvironment at port 5005
[INFO] Connected new brain: VectorNavigator?team=0
[INFO] Connected to Unity environment with package version 2.1.0-exp.1 and communication version 1.5.0
Created UnityEnvironment at port 5005
[INFO] Connected new brain: VectorNavigator?team=0
[INFO] Connected to Unity environment with package version 2.1.0-exp.1 and communication version 1.5.0
Created UnityEnvironment at port 5005
[INFO] Connected new brain: VectorNavigator?team=0
[INFO] Connected to Unity environment with package version 2.1.0-exp.1 and communication version 1.5.0
Created UnityEnvironment at port 5005
[INFO] Connected new brain: VectorNavigator?team=0
[INFO] Connected to Unity environment with package version 2.1.0-exp.1 and communication version 1.5.0
Created UnityEnvironment at port 5005
[I

In [6]:
# test for appending to cupy memory

num_episodes=10
num_steps=10
num_actions = 3

rc_stats = [[0.0]*2]
for i in [0,1,2]:
    env_config["observation_mode"]=i
    
    env = NavSimGymEnv(env_config)
    memory_observation_space_shapes = []
    for item in env.observation_space_shapes:
        if len(item) == 3:  
            # means its an image in HWC
            # we pass the shapes as CWH
            memory_observation_space_shapes.append(
                (item[2], item[1], item[0]))
        else:
            memory_observation_space_shapes.append(item)
    memory_capacity = 100

    memory = CupyMemory(capacity=memory_capacity,
                        state_shapes=memory_observation_space_shapes,
                        action_shape=env.action_space.shape,
                        seed=env_config['seed']
                    )
    
    memory.info()
    for i in range(0,num_episodes):
        s = env.reset()
        s = s_hwc_to_chw(s)
        episode_done=False
        while not episode_done:
            a=env.action_space.sample()
            #step_res[5] = self.rc.snapshot()  # s5
            s_, r, episode_done, info = env.step(a)
            #step_res[6] = self.rc.snapshot()  # s6
            s_ = s_hwc_to_chw(s_)
            memory.append(s=s, a=a, s_=s_, r=r,d=float(episode_done))
    memory.info()
    env.close()
    del env

[autoreload of navsim.executor.executor failed: Traceback (most recent call last):
  File "/opt/conda/envs/navsim/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/opt/conda/envs/navsim/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/opt/conda/envs/navsim/lib/python3.8/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/opt/conda/envs/navsim/lib/python3.8/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 604, in _exec
  File "<frozen importlib._bootstrap_external>", line 844, in exec_module
  File "<frozen importlib._bootstrap_external>", line 981, in get_code
  File "<frozen importlib._bootstrap_external>", line 911, in source_to_code
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/data/afan

[INFO] Connected to Unity environment with package version 2.1.0-exp.1 and communication version 1.5.0
Created UnityEnvironment at port 5005
[INFO] Connected new brain: VectorNavigator?team=0
-----------
Memory Info
-----------
module: CupyMemory 
capacity: 100 
size: 0 
current pointer: 0 
seed: 321 
Shapes: 
s: ["(100, 13) of <class 'cupy._core.core.ndarray'>"] 
a: (100, 3) <class 'cupy._core.core.ndarray'> 
r: (100, 1) <class 'cupy._core.core.ndarray'> 
s_: ["(100, 13) of <class 'cupy._core.core.ndarray'>"] 
d: (100, 1) <class 'cupy._core.core.ndarray'> 
----------------------------------------
-----------
Memory Info
-----------
module: CupyMemory 
capacity: 100 
size: 82 
current pointer: 82 
seed: 321 
Shapes: 
s: ["(100, 13) of <class 'cupy._core.core.ndarray'>"] 
a: (100, 3) <class 'cupy._core.core.ndarray'> 
r: (100, 1) <class 'cupy._core.core.ndarray'> 
s_: ["(100, 13) of <class 'cupy._core.core.ndarray'>"] 
d: (100, 1) <class 'cupy._core.core.ndarray'> 
---------------------

In [7]:
# test for appending to numpy memory

num_episodes=10
num_steps=10
num_actions = 3

for i in [0,1,2]:
    env_config["observation_mode"]=i
    
    env = NavSimGymEnv(env_config)
    memory_observation_space_shapes = []
    for item in env.observation_space_shapes:
        if len(item) == 3:  
            # means its an image in HWC
            # we pass the shapes as CWH
            memory_observation_space_shapes.append(
                (item[2], item[1], item[0]))
        else:
            memory_observation_space_shapes.append(item)
    memory_capacity = 100

    memory = NumpyMemory(capacity=memory_capacity,
                        state_shapes=memory_observation_space_shapes,
                        action_shape=env.action_space.shape,
                        seed=env_config['seed']
                    )
    
    memory.info()
    for i in range(0,num_episodes):
        s = env.reset()
        s = s_hwc_to_chw(s)
        episode_done=False
        while not episode_done:
            a=env.action_space.sample()
            #step_res[5] = self.rc.snapshot()  # s5
            s_, r, episode_done, info = env.step(a)
            #step_res[6] = self.rc.snapshot()  # s6
            s_ = s_hwc_to_chw(s_)
            memory.append(s=s, a=a, s_=s_, r=r,d=float(episode_done))
    memory.info()
    env.close()
    del env

[autoreload of navsim.executor.executor failed: Traceback (most recent call last):
  File "/opt/conda/envs/navsim/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/opt/conda/envs/navsim/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/opt/conda/envs/navsim/lib/python3.8/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/opt/conda/envs/navsim/lib/python3.8/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 604, in _exec
  File "<frozen importlib._bootstrap_external>", line 844, in exec_module
  File "<frozen importlib._bootstrap_external>", line 981, in get_code
  File "<frozen importlib._bootstrap_external>", line 911, in source_to_code
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/data/afan

[INFO] Connected to Unity environment with package version 2.1.0-exp.1 and communication version 1.5.0
Created UnityEnvironment at port 5005
[INFO] Connected new brain: VectorNavigator?team=0
-----------
Memory Info
-----------
module: NumpyMemory 
capacity: 100 
size: 0 
current pointer: 0 
seed: 321 
Shapes: 
s: ["(100, 13) of <class 'numpy.ndarray'>"] 
a: (100, 3) <class 'numpy.ndarray'> 
r: (100, 1) <class 'numpy.ndarray'> 
s_: ["(100, 13) of <class 'numpy.ndarray'>"] 
d: (100, 1) <class 'numpy.ndarray'> 
----------------------------------------
-----------
Memory Info
-----------
module: NumpyMemory 
capacity: 100 
size: 82 
current pointer: 82 
seed: 321 
Shapes: 
s: ["(100, 13) of <class 'numpy.ndarray'>"] 
a: (100, 3) <class 'numpy.ndarray'> 
r: (100, 1) <class 'numpy.ndarray'> 
s_: ["(100, 13) of <class 'numpy.ndarray'>"] 
d: (100, 1) <class 'numpy.ndarray'> 
----------------------------------------
[INFO] Connected to Unity environment with package version 2.1.0-exp.1 and com

In [8]:
env.close()
del env

NameError: name 'env' is not defined